In [1]:
import zipfile
import os

# Définir le chemin du fichier ZIP et le répertoire de destination
zip_file_path = '/home/alexis/Downloads/NEWCAM.zip'
extract_to_path = '/home/alexis/Downloads/NEWCAM'

# Décompresser le fichier ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

# Lister les fichiers extraits
extracted_files = os.listdir(extract_to_path)
extracted_files




['annotations.xml']

In [2]:
# Explorer le contenu extrait
for root, dirs, files in os.walk(extract_to_path):
    for file in files:
        print(os.path.join(root, file))


/home/alexis/Downloads/NEWCAM/annotations.xml


In [3]:
import xml.etree.ElementTree as ET

# Chemin du fichier XML
xml_file_path = '/home/alexis/Downloads/NEWCAM/annotations.xml'

# Lire et analyser le fichier XML
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Explorer la structure du fichier XML
structure = []
for child in root:
    structure.append(child.tag)

structure

['version',
 'meta',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track',
 'track']

In [4]:
# Lire et analyser les annotations du fichier XML
annotations = []

for track in root.findall('track'):
    label = track.attrib['label']
    for box in track.findall('box'):
        frame_id = int(box.attrib['frame'])
        xtl = float(box.attrib['xtl'])
        ytl = float(box.attrib['ytl'])
        xbr = float(box.attrib['xbr'])
        ybr = float(box.attrib['ybr'])
        width = xbr - xtl
        height = ybr - ytl
        x_center = xtl + width / 2
        y_center = ytl + height / 2

        annotation = {
            'frame_id': frame_id,
            'label': label,
            'x_center': x_center,
            'y_center': y_center,
            'width': width,
            'height': height
        }
        annotations.append(annotation)

annotations[:5]  # Afficher les premières annotations pour vérification


[{'frame_id': 0,
  'label': 'diver',
  'x_center': 372.675,
  'y_center': 125.515,
  'width': 184.11,
  'height': 109.03},
 {'frame_id': 1,
  'label': 'diver',
  'x_center': 373.0,
  'y_center': 124.5,
  'width': 184.0,
  'height': 109.0},
 {'frame_id': 2,
  'label': 'diver',
  'x_center': 372.0,
  'y_center': 122.5,
  'width': 184.0,
  'height': 109.0},
 {'frame_id': 3,
  'label': 'diver',
  'x_center': 371.0,
  'y_center': 120.5,
  'width': 184.0,
  'height': 109.0},
 {'frame_id': 4,
  'label': 'diver',
  'x_center': 370.0,
  'y_center': 120.5,
  'width': 184.0,
  'height': 109.0}]

In [33]:
import os
import xml.etree.ElementTree as ET

# Définir les chemins
xml_file_path = '/home/alexis/Downloads/NEWCAM/annotations.xml'
yolo_annotations_path = '/home/alexis/Downloads/NEWCAM/labels'
os.makedirs(yolo_annotations_path, exist_ok=True)

# Lire et analyser les annotations du fichier XML
annotations = []
tree = ET.parse(xml_file_path)
root = tree.getroot()

for track in root.findall('track'):
    label = track.attrib['label']
    for box in track.findall('box'):
        frame_id = int(box.attrib['frame'])
        xtl = float(box.attrib['xtl'])
        ytl = float(box.attrib['ytl'])
        xbr = float(box.attrib['xbr'])
        ybr = float(box.attrib['ybr'])

        # Check if the coordinates are within the valid range
        if xtl < 0 or ytl < 0 or xbr < 0 or ybr < 0:
            continue

        width = xbr - xtl
        height = ybr - ytl
        x_center = xtl + width / 2
        y_center = ytl + height / 2

        annotation = {
            'frame_id': frame_id,
            'label': label,
            'x_center': x_center,
            'y_center': y_center,
            'width': width,
            'height': height
        }
        annotations.append(annotation)

# Mapper les étiquettes à des IDs numériques
label_map = {'diver': 0, 'fish': 1, 'dolphin': 2}  # Add 'dolphin' label to the label_map dictionary

# Créer les fichiers d'annotation YOLO
for annotation in annotations:
    frame_id = annotation['frame_id']
    label_id = label_map[annotation['label']]
    x_center = annotation['x_center']
    y_center = annotation['y_center']
    width = annotation['width']
    height = annotation['height']

    # Nom du fichier d'annotation
    annotation_file = os.path.join(yolo_annotations_path, f'{frame_id:06d}.txt')

    # Écrire l'annotation dans le fichier
    with open(annotation_file, 'a') as f:
        f.write(f'{label_id} {x_center} {y_center} {width} {height}\n')

print("Annotations YOLO créées avec succès.")


Annotations YOLO créées avec succès.


In [36]:
import os

# Directory path
labels_folder = '/home/alexis/Desktop/New_caméra/Custom_dataset/train/labels/'

# Get the list of files in the directory
files = os.listdir(labels_folder)

# Sort the files in ascending order
files.sort(reverse = False)

# Get the label value from the last file in the directory
label_value = int(files[-1].split('.')[0].zfill(6))
label_value


9898

In [25]:
import cv2
import os

# Path to the video file
video_path = '/home/alexis/Desktop/New_caméra/Exp_01_Jun_2024_1145_cam1-4.mp4'

# Directory to save the extracted frames
output_dir = '/home/alexis/Desktop/New_caméra/frames/'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Open the video file
video = cv2.VideoCapture(video_path)

# Read the first frame
success, frame = video.read()
frame_count = 0

# Loop through the video frames
while success and frame_count <= label_value:
    # Save the frame as a JPG image
    frame_path = os.path.join(output_dir, f'{frame_count:06d}.jpg')
    cv2.imwrite(frame_path, frame)

    # Read the next frame
    success, frame = video.read()
    frame_count += 1

# Release the video file
video.release()


KeyboardInterrupt: 

In [37]:
import os
import random
import shutil

# Define the paths
train_images_dir = '/home/alexis/Desktop/New_caméra/Custom_dataset/train/images'
train_labels_dir = '/home/alexis/Desktop/New_caméra/Custom_dataset/train/labels'
val_images_dir = '/home/alexis/Desktop/New_caméra/Custom_dataset/val/images'
val_labels_dir = '/home/alexis/Desktop/New_caméra/Custom_dataset/val/labels'

# Create the val directories if they don't exist
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Get the list of image files in the train directory
labels_files = os.listdir(train_labels_dir)

# Calculate the number of images to move
num_images_to_move = int(len(labels_files) * 0.05)

# Randomly select the images to move
labels_to_move = random.sample(labels_files, num_images_to_move)

# Move the selected images and labels to the val directory
for label_file in labels_to_move:
    # Move the image file
    image_file = label_file.replace('.txt', '.jpg')
    src_image_path = os.path.join(train_images_dir, image_file)
    dest_image_path = os.path.join(val_images_dir, image_file)
    shutil.move(src_image_path, dest_image_path)

    # Move the corresponding label file
    src_label_path = os.path.join(train_labels_dir, label_file)
    dest_label_path = os.path.join(val_labels_dir, label_file)
    shutil.move(src_label_path, dest_label_path)

# Print the number of images moved
print(f"{num_images_to_move} images and labels moved from train to val directory.")


134 images moved from train to val directory.


In [31]:
# Delete the images without labels
for image_file in os.listdir(train_images_dir):
    label_file = image_file.replace('.jpg', '.txt')
    if label_file not in labels_files:
        image_path = os.path.join(train_images_dir, image_file)
        os.remove(image_path)

In [38]:
import yaml

# Define the classes
classes = ['dolphin', 'diver', 'fish']

# Define the data dictionary
data = {
    'train': '/home/alexis/Desktop/New_caméra/Custom_dataset/train/images',
    'val': '/home/alexis/Desktop/New_caméra/Custom_dataset/val/images',
    'nc': len(classes),
    'names': classes
}

# Write the data dictionary to the data.yaml file
with open('/home/alexis/Desktop/New_caméra/Custom_dataset/data.yaml', 'w') as f:
    yaml.dump(data, f)
